In [44]:
# Check if Temperature is above 95º and humidity is below 30%, if so fire risk is True and viceversa
# Then place location and fire risk boolean value into a csv file.

import requests
import csv

# Latitude and Longitude for San Francisco, CA
latitude = 37.77056
longitude = -122.42694

# Construct the endpoint URL
endpoint = f'https://api.weather.gov/points/{latitude},{longitude}'

# Define headers
headers = {
    'User-Agent': 'weather-app',
    'Accept': 'application/geo+json'
}

try:
    # Step 1: Make the API request to get grid points
    response = requests.get(endpoint, headers=headers)

    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        # Extract the observation stations URL
        observation_stations_url = data['properties']['observationStations']

        # Step 2: Make the API request to get observation stations
        response_stations = requests.get(observation_stations_url, headers=headers)

        if response_stations.status_code == 200:
            stations_data = response_stations.json()
            if 'features' in stations_data and len(stations_data['features']) > 0:
                station_id = stations_data['features'][0]['properties']['stationIdentifier']
                print(f"Station ID: {station_id}")

                # Step 3: Construct the URL to get the latest observation for the station
                latest_observation_url = f'https://api.weather.gov/stations/{station_id}/observations/latest'

                # Step 4: Make the API request to get the latest observation
                response_observation = requests.get(latest_observation_url, headers=headers)

                if response_observation.status_code == 200:
                    observation_data = response_observation.json()

                    # Extract relevant weather information
                    properties = observation_data['properties']
                    temperature_c = properties['temperature']['value']  # Temperature in Celsius
                    temperature_f = (temperature_c * 9/5) + 32  # Temperature in Fahrenheit
                    humidity = properties['relativeHumidity']['value']  # Relative humidity in %

                    # Check conditions for fire risk
                    above_90 = temperature_f > 90
                    fire_risk = above_90 and humidity < 30

                    # Print weather details
                    print(f"Weather in San Francisco:")
                    print(f"Temperature: {temperature_c:.2f}°C / {temperature_f:.2f}°F")
                    print(f"Humidity: {humidity:.2f}%")
                    print(f"Fire Risk: {'True' if fire_risk else 'False'}")

                    # Prepare data to write to CSV
                    weather_data = [
                        ['Location', 'Fire Risk'],
                        ['San Francisco, CA', fire_risk]
                    ]

                    # Write data to CSV file
                    with open('weather_alerts.csv', mode='w', newline='') as file:
                        writer = csv.writer(file)
                        writer.writerows(weather_data)

                    print("Weather alert data written to weather_alerts.csv")

                else:
                    print(f"Error fetching latest observation: {response_observation.status_code}")
                    print(response_observation.json())
            else:
                print("No observation stations found.")
        else:
            print(f"Error fetching observation stations: {response_stations.status_code}")
            print(response_stations.json())
    else:
        print(f"Error fetching grid points: {response.status_code}")
        print(response.json())

except requests.exceptions.RequestException as e:
    print(f"Request error: {e}")


Station ID: SFOC1
Weather in San Francisco:
Temperature: 18.09°C / 64.56°F
Humidity: 61.98%
Fire Risk: False
Weather alert data written to weather_alerts.csv


---

## Testing Below

---


In [32]:
import requests

# Latitude and Longitude for San Francisco, CA
latitude = 37.77056
longitude = -122.42694

# Construct the endpoint URL
endpoint = f'https://api.weather.gov/points/{latitude},{longitude}'

# Define headers
headers = {
    'User-Agent': 'weather-app',
    'Accept': 'application/geo+json'
}

# Step 1: Make the API request to get grid points
response = requests.get(endpoint, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()
    # Extract the observation stations URL
    observation_stations_url = data['properties']['observationStations']
    
    # Step 2: Make the API request to get observation stations
    response_stations = requests.get(observation_stations_url, headers=headers)
    
    if response_stations.status_code == 200:
        stations_data = response_stations.json()
        if 'features' in stations_data and len(stations_data['features']) > 0:
            station_id = stations_data['features'][0]['properties']['stationIdentifier']
            print(f"Station ID: {station_id}")
            
            # Step 3: Construct the URL to get the latest observation for the station
            latest_observation_url = f'https://api.weather.gov/stations/{station_id}/observations/latest'
            
            # Step 4: Make the API request to get the latest observation
            response_observation = requests.get(latest_observation_url, headers=headers)
            
            if response_observation.status_code == 200:
                observation_data = response_observation.json()
                
                # Extract relevant weather information
                properties = observation_data['properties']
                temperature = properties['temperature']['value']  # Temperature in Celsius
                wind_speed = properties['windSpeed']['value']  # Wind speed in m/s
                humidity = properties['relativeHumidity']['value']  # Relative humidity in %
                weather = properties['textDescription']  # Weather description

                # Convert temperature to Fahrenheit if it's not None
                temperature_f = (temperature * 9/5) + 32 if temperature is not None else None

                # Print the weather information with checks for None values
                print(f"Weather in San Francisco:")
                if temperature is not None and temperature_f is not None:
                    print("High fire risk: temperatures above 90°F (32°C) with dry conditions.")
                    print(f"Temperature: {temperature:.2f}°C / {temperature_f:.2f}°F")
                else:
                    print("Temperature: Not available")

                if wind_speed is not None:
                    print(f"Wind Speed: {wind_speed:.2f} m/s")
                else:
                    print("Wind Speed: Not available")

                if humidity is not None:
                    print(f"Humidity: {humidity:.2f}%")
                else:
                    print("Humidity: Not available")

                if weather is not None:
                    print(f"Conditions: {weather}")
                else:
                    print("Conditions: Not available")
            else:
                print(f"Error fetching latest observation: {response_observation.status_code}")
                print(response_observation.json())
        else:
            print("No observation stations found.")
    else:
        print(f"Error fetching observation stations: {response_stations.status_code}")
        print(response_stations.json())
else:
    print(f"Error fetching grid points: {response.status_code}")
    print(response.json())


Station ID: SFOC1
Weather in San Francisco:
High fire risk: temperatures above 90°F (32°C) with dry conditions.
Temperature: 18.09°C / 64.56°F
Wind Speed: Not available
Humidity: 61.98%
Conditions: 


In [16]:
import requests

# Define the NOAA Alerts API endpoint
alerts_endpoint = 'https://api.weather.gov/alerts'

# Define headers for the request
headers = {
    'User-Agent': 'weather-app',
    'Accept': 'application/geo+json'
}

try:
    # Make the API request to fetch alerts
    response = requests.get(alerts_endpoint, headers=headers)

    if response.status_code == 200:
        data = response.json()
        
        # Check if there are any alerts
        if 'features' in data and len(data['features']) > 0:
            print("Alerts:")
            for alert in data['features']:
                properties = alert['properties']
                event = properties['event']
                headline = properties['headline']
                severity = properties['severity']
                print(f"- Event: {event}")
                print(f"  Headline: {headline}")
                print(f"  Severity: {severity}")
                print()
        else:
            print("No alerts found.")

    else:
        print(f"Error fetching alerts: {response.status_code}")

except requests.exceptions.RequestException as e:
    print(f"Request error: {e}")


Alerts:
- Event: Special Weather Statement
  Headline: Special Weather Statement issued June 29 at 6:24PM CDT by NWS Amarillo TX
  Severity: Moderate

- Event: Severe Thunderstorm Warning
  Headline: Severe Thunderstorm Warning issued June 29 at 6:24PM CDT until June 29 at 7:15PM CDT by NWS Tulsa OK
  Severity: Severe

- Event: Special Weather Statement
  Headline: Special Weather Statement issued June 29 at 7:23PM EDT by NWS Gaylord MI
  Severity: Moderate

- Event: Severe Thunderstorm Warning
  Headline: Severe Thunderstorm Warning issued June 29 at 7:23PM EDT until June 29 at 7:30PM EDT by NWS Wilmington OH
  Severity: Severe

- Event: Severe Thunderstorm Warning
  Headline: Severe Thunderstorm Warning issued June 29 at 7:22PM EDT until June 29 at 7:45PM EDT by NWS Charleston WV
  Severity: Severe

- Event: Severe Thunderstorm Warning
  Headline: Severe Thunderstorm Warning issued June 29 at 6:22PM CDT until June 29 at 6:45PM CDT by NWS Norman OK
  Severity: Severe

- Event: Severe 

In [24]:
# Same code as above, but filtered for San Francisco CA
import requests

# Define the NOAA Alerts API endpoint
alerts_endpoint = 'https://api.weather.gov/alerts'

# Define headers for the request
headers = {
    'User-Agent': 'weather-app',
    'Accept': 'application/geo+json'
}

try:
    # Make the API request to fetch alerts
    response = requests.get(alerts_endpoint, headers=headers)

    if response.status_code == 200:
        data = response.json()
        
        # Check if there are any alerts
        if 'features' in data and len(data['features']) > 0:
            print("Alerts for San Francisco, CA:")
            for alert in data['features']:
                properties = alert['properties']
                event = properties['event']
                headline = properties['headline']
                severity = properties['severity']
                
                # Ensure headline is not None and ends with "San Francisco CA"
                if headline and isinstance(headline, str) and headline.endswith("San Francisco CA"):
                    print(f"- Event: {event}")
                    print(f"  Headline: {headline}")
                    print(f"  Severity: {severity}")
                    print()
        
        else:
            print("No alerts found.")

    else:
        print(f"Error fetching alerts: {response.status_code}")

except requests.exceptions.RequestException as e:
    print(f"Request error: {e}")


Alerts for San Francisco, CA:
- Event: Small Craft Advisory
  Headline: Small Craft Advisory issued June 29 at 2:05PM PDT until July 1 at 9:00AM PDT by NWS San Francisco CA
  Severity: Minor

- Event: Small Craft Advisory
  Headline: Small Craft Advisory issued June 29 at 2:05PM PDT until June 30 at 3:00PM PDT by NWS San Francisco CA
  Severity: Minor

- Event: Small Craft Advisory
  Headline: Small Craft Advisory issued June 29 at 2:05PM PDT until July 1 at 3:00AM PDT by NWS San Francisco CA
  Severity: Minor

- Event: Small Craft Advisory
  Headline: Small Craft Advisory issued June 29 at 2:05PM PDT until July 1 at 9:00AM PDT by NWS San Francisco CA
  Severity: Minor

- Event: Small Craft Advisory
  Headline: Small Craft Advisory issued June 29 at 2:05PM PDT until July 1 at 9:00AM PDT by NWS San Francisco CA
  Severity: Minor

- Event: Small Craft Advisory
  Headline: Small Craft Advisory issued June 29 at 2:05PM PDT until June 30 at 9:00PM PDT by NWS San Francisco CA
  Severity: Min